In [1]:
import pandas as pd
import numpy as np
from yahoo_fin.stock_info import get_data
import statsmodels.formula.api as smf
from pypfopt.efficient_frontier import EfficientFrontier
import datetime

### Fetch historical data of Jan 2018 - Dec 2020. Test date starts from Jan 2020. 
### Reason why to choose Jan 2018 as the START is all the 72 CCs ICO before 2018, which means ALL data after 2018 is accessible. 
### Drawbacks: Jan 2018 is the first peak in CC history which may result in low Sharpe Ratio and Return of this strategy (in further research, try to include data from 2017 and solve data missing of some CCs.

In [2]:
START = TRAIN_START = datetime.datetime(2018,1,1)
TEST_START = datetime.datetime(2020,1,5)
END = datetime.datetime(2020,12,31)

### Download cryptocurrency(CC) market 100 index from [Bitwise](https://www.bitwiseinvestments.com/indexes/Bitwise-100) which covers 93% of the CC market.

In [3]:
CC100_Index = pd.read_csv("indexReturns-BITW100.csv")
CC100_Index['date'] = pd.to_datetime(CC100_Index['date']).dt.tz_localize(None)
CC100_Index = CC100_Index.sort_values('date').set_index('date')
CC100_Index = CC100_Index.truncate(before = START,after = END)
CC100_Index = CC100_Index.resample('W').last()[:-1]
CC100_Return = np.log(CC100_Index/ CC100_Index.shift(1)).fillna(axis = 0, method ='bfill').rename(columns = {"MarketIndex":"MarketReturn"})
CC100 = pd.concat([CC100_Index, CC100_Return],axis =1)

In [4]:
CC100

,MarketIndex,MarketReturn
date,,
2018-01-07,34027.35,-0.108315
2018-01-14,30534.28,-0.108315
2018-01-21,25761.58,-0.169966
2018-01-28,22724.66,-0.125434
2018-02-04,18134.05,-0.225659
...,...,...
2020-11-29,18386.98,-0.001030
2020-12-06,19257.57,0.046261
2020-12-13,19002.90,-0.013313


In [5]:
TEST_TimeList = CC100.loc[TEST_START:].index

#### VET Deleted due to late ICO (in Aug 2018)

In [6]:
CC_string = "ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX"
CC_temp_list = CC_string.split(",")
CC_list = [i+'-USD' for i in CC_temp_list]

### Fetching the following data may take some time. Thus, "data_copy" is used to protect "data_all" from being changed.

In [7]:
data_all = pd.DataFrame()
for i in CC_list:
    data = get_data(i,start_date = START,end_date = END).resample('W').last()[:-1]
    data_all = pd.concat([data_all,data])
data_all.index.names = ['date']
data_all

,open,high,low,close,adjclose,volume,ticker
date,,,,,,,
2018-01-07,1.024150,1.043960,0.991391,1.008520,1.008520,226067008.0,ADA-USD
2018-01-14,0.881961,0.881961,0.749514,0.802262,0.802262,160712000.0,ADA-USD
2018-01-21,0.706391,0.706391,0.590793,0.613163,0.613163,847731008.0,ADA-USD
2018-01-28,0.620560,0.681946,0.620560,0.670286,0.670286,325145984.0,ADA-USD
2018-02-04,0.433865,0.480140,0.369823,0.379303,0.379303,814753984.0,ADA-USD
...,...,...,...,...,...,...,...
2020-11-29,0.415439,0.419485,0.405365,0.414702,0.414702,37263535.0,ZRX-USD
2020-12-06,0.407349,0.410595,0.394628,0.400748,0.400748,33594446.0,ZRX-USD
2020-12-13,0.392727,0.403993,0.386825,0.396126,0.396126,31834561.0,ZRX-USD


### Example:  "date" is the first date of "TEST_TimeList".  Find the top10 liquid CCs.

In [8]:
data_copy = data_all.copy()
date = TEST_TimeList[0]
top10_liquid = data_copy.loc[date].sort_values('volume',ascending= False).iloc[:10]

data_multiIndex = data_copy.copy().reset_index().set_index(['ticker','date'])

price_top10_liquid = pd.DataFrame()
for ticker in top10_liquid['ticker']:
    price_top10_liquid[ticker] = data_multiIndex.loc[ticker][:date]['adjclose']
returns_top10_liquid = np.log(price_top10_liquid/ price_top10_liquid.shift(1)).fillna(axis = 0, method ='bfill').rename(columns=lambda s: s.replace('-USD',''))

returns_top10withMarket =returns_top10_liquid.copy()
returns_top10withMarket['MarketReturn'] = CC100.loc[:date]['MarketReturn']

returns_top10withMarket

,BTC,ETH,LTC,EOS,BCH,XRP,TRX,ETC,DASH,NEO,MarketReturn
date,,,,,,,,,,,
2018-01-07,-0.179364,0.169936,-0.189422,0.070436,-0.087528,-0.596012,-0.856508,0.051694,-0.237121,0.509084,-0.108315
2018-01-14,-0.179364,0.169936,-0.189422,0.070436,-0.087528,-0.596012,-0.856508,0.051694,-0.237121,0.509084,-0.108315
2018-01-21,-0.171624,-0.264060,-0.225865,0.000022,-0.362671,-0.299104,-0.174787,-0.348330,-0.214530,-0.262332,-0.169966
2018-01-28,0.015924,0.171556,0.017925,0.111478,-0.017253,0.014074,-0.083322,0.081614,-0.007399,0.148984,-0.125434
2018-02-04,-0.353456,-0.400651,-0.269781,-0.556996,-0.404423,-0.539891,-0.546152,-0.447189,-0.389946,-0.347834,-0.225659
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-08,0.018688,0.000522,-0.037548,-0.006832,-0.008424,0.022389,-0.077945,-0.004749,-0.020527,-0.021587,0.020678
2019-12-15,-0.056012,-0.055383,-0.048970,-0.069453,-0.033160,-0.052721,-0.050459,-0.026017,-0.022481,-0.014102,-0.053090
2019-12-22,0.049013,-0.078028,-0.041807,-0.012662,-0.052994,-0.104156,0.061178,0.095030,-0.140550,-0.005366,0.013306


### CAPM Regressoin

In [9]:
regression = pd.DataFrame(columns=['beta', 'rse'])
for ticker in top10_liquid['ticker']:
    ticker = ticker.split('-')[0]
    CAPM_model = smf.ols(formula = ticker+' ~ MarketReturn', data = returns_top10withMarket)
    CAPM_fit = CAPM_model.fit()
    beta = CAPM_fit.params['MarketReturn']
    rse = CAPM_fit.bse['MarketReturn']
    dict = {'beta': beta, 'rse': rse} 
    df = pd.DataFrame(dict, index = [ticker])
    regression = regression.append(df)
regression

,beta,rse
BTC,0.851921,0.045235
ETH,0.987236,0.077407
LTC,0.913333,0.076613
EOS,1.085980,0.094061
BCH,1.465015,0.102978
XRP,0.948610,0.102517
TRX,1.215633,0.130651
ETC,0.982677,0.090277
DASH,0.973757,0.070729
NEO,0.999878,0.108443


### Calculate the Expected retrun using mean Market Return
## Qusestion: should use the latest market return OR the mean to forecast the next week's return (Expected Return)?
## Things to do: understand the detailed algorithm of the CAPM regression

In [10]:
data = returns_top10withMarket.mean()['MarketReturn']*regression['beta']
ExpectedReturns = pd.DataFrame(data)
ExpectedReturns.columns = ['returns']
ExpectedReturns

,returns
BTC,-0.013372
ETH,-0.015496
LTC,-0.014336
EOS,-0.017045
BCH,-0.022995
XRP,-0.014889
TRX,-0.019080
ETC,-0.015424
DASH,-0.015284
NEO,-0.015694


In [11]:
CovarianceMatrix = returns_top10_liquid.cov()
CovarianceMatrix

,BTC,ETH,LTC,EOS,BCH,XRP,TRX,ETC,DASH,NEO
BTC,0.012083,0.011750,0.011832,0.012668,0.017499,0.011969,0.015713,0.011305,0.012192,0.011151
ETH,0.011750,0.020536,0.015058,0.018941,0.024000,0.013867,0.014505,0.017414,0.015778,0.020647
LTC,0.011832,0.015058,0.018562,0.016531,0.022015,0.014381,0.018034,0.015414,0.015174,0.014211
EOS,0.012668,0.018941,0.016531,0.026972,0.025009,0.015353,0.020790,0.017837,0.017158,0.020480
BCH,0.017499,0.024000,0.022015,0.025009,0.041782,0.020776,0.024690,0.023104,0.023517,0.022568
XRP,0.011969,0.013867,0.014381,0.015353,0.020776,0.025575,0.025061,0.014230,0.016559,0.009925
TRX,0.015713,0.014505,0.018034,0.020790,0.024690,0.025061,0.041748,0.016458,0.020289,0.010806
ETC,0.011305,0.017414,0.015414,0.017837,0.023104,0.014230,0.016458,0.023288,0.015196,0.018892
DASH,0.012192,0.015778,0.015174,0.017158,0.023517,0.016559,0.020289,0.015196,0.018881,0.014917
NEO,0.011151,0.020647,0.014211,0.020480,0.022568,0.009925,0.010806,0.018892,0.014917,0.028525


In [12]:
ExpectedReturns['returns']

BTC    -0.013372
ETH    -0.015496
LTC    -0.014336
EOS    -0.017045
BCH    -0.022995
XRP    -0.014889
TRX    -0.019080
ETC    -0.015424
DASH   -0.015284
NEO    -0.015694
Name: returns, dtype: float64

In [13]:
ef1 = EfficientFrontier(ExpectedReturns['returns'], CovarianceMatrix, weight_bounds=(0,1))
weights = ef1.max_sharpe(risk_free_rate= 0.0)
weights = ef1.clean_weights()

weights

OptimizationError: Please check your objectives/constraints or use a different solver.